# Data preparation
This notebook extracts and renames columns from csv files at each time point: baseline, 1_year, 2_year, 3_year. The configurations for which columns to extract are located in `specifications.py` as a named tuple - this is where to look if you'd like to make any changes. Minimal data processing occurs at this stage. It is limited to removing void answers, excluding cognitive task variables with unacceptable performance (indicated by flags), and deriving a few mean or ratio fields. These can be found `data_preparation.py`. 

This google doc outlines which scales have been extracted and calculated: https://docs.google.com/document/d/1-TspbHm00e0Gw14p7WJBpR-8cJYCTR_BLB0qNnVFDz0/edit#

In [ ]:
import pandas as pd
import numpy as np
from datetime import date
from data_preparation import get_cognitive_tasks, get_demographics, get_outcomes, get_neuraldata

In [2]:
raw_data = '../raw/'
derived_data = '../derived/'

In [4]:
version_old = '20211118'
version_new = date.today().strftime('%Y%m%d')
cct_df = pd.read_csv(f'{raw_data}cct01.txt', delimiter='\t')
subjects = cct_df.subjectkey.unique()[1:]

In [ ]:
for period in ['baseline', '1_year', '2_year', '3_year']:
    tasks = get_cognitive_tasks(subjects=subjects, period=period, version=version_new, load_latest=False)
    demo = get_demographics(subjects=subjects, period=period, version=version_new, load_latest=False)
    neural = get_neuraldata(subjects=subjects, period=period, version=version_new, load_latest=False)
    outcomes = get_outcomes(subjects=subjects, period=period, version=version_new, load_latest=False)
    
    data = pd.DataFrame(subjects, columns=['subject'])
    data = pd.merge(data, tasks, on=['subject'], how='left')
    data = pd.merge(data, demo, on=['subject'], how='left')
    data = pd.merge(data, neural, on=['subject'], how='left')
    data = pd.merge(data, outcomes, on=['subject'], how='left')
    print(data.shape)
    
    data.to_csv(f'{derived_data}{version_new}/data_{period}.csv', index_label=False)
    
    if period == "baseline":
        baseline = data.copy()
        baseline.insert(1, 'time', 0)
    elif period == '1_year':
        one_year = data.copy()
        one_year.insert(1, 'time', 1)
    elif period == '2_year':
        two_year = data.copy()
        two_year.insert(1, 'time', 2)  
    else:
        three_year = data.copy()
        three_year.insert(1, 'time', 3) 

# Panel data

In [35]:
merged = baseline.append(one_year)
merged = merged.append(two_year)
merged = merged.append(three_year)
merged = merged.sort_values(by=['subject', 'time'])
merged.to_csv(f'{derived_data}{version_new}/task_demo_outcomes_panel.csv')

In [ ]:
merged.shape

# Aggregated data

In [36]:
aggregated = merged.groupby(['subject', 'sex']).mean().reset_index(drop=False)
aggregated.drop(columns=['time'], inplace=True)
aggregated.to_csv(f'{derived_data}{version_new}/task_demo_outcomes_aggregated.csv')

In [ ]:
aggregated.shape